In [1]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import librosa.display as ld
import pyaudio
import wave
import sys
import hashlib
import mysql.connector
import time
import os

In [2]:
selection=False
while not selection:
    choice=int(input("Select your choice: 1. Read a song into the database 2. Read a recording  3.Exit "))
    selection=(choice==1 or choice==2 or choice==3)
    if not selection:
        print("Invalid choice. Try again")

Select your choice: 1. Read a song into the database 2. Read a recording  3.Exit 3


In [3]:
if choice==1:
    start_time = time.time()

    mydb = mysql.connector.connect(
      host="localhost",
      user="<Enter user name here>",
      passwd="<Enter password here>",
      database="songsdb"
    )    
    nameofsong=input("Enter song name")
    filename=nameofsong+".wav"
    
    mycursor=mydb.cursor()
    sub=[nameofsong]
    clu="INSERT INTO ref(Song_name) VALUES (%s)"
    mycursor.execute(clu,sub)
    mydb.commit()
    
    recarr,sr=librosa.load(filename)
    
    def ftransform(signal,title,sr,f_ratio):
        ft=np.fft.fft(signal)
        mag_spec=np.abs(ft)

        plt.figure(figsize=(12,5))
        frequency=np.linspace(0,sr,len(mag_spec))
        num_freq_bins=int(len(frequency)*f_ratio)
        plt.plot(frequency[:num_freq_bins],mag_spec[:num_freq_bins])
        plt.xlabel('Frequency')
        plt.title(title)

        plt.show()
    
    ftransform(recarr,filename,sr,0.5)
    
    FRAME_SIZE=2048
    HOP_SIZE=512
    
    stft_recarr=librosa.stft(recarr,n_fft=FRAME_SIZE,hop_length=HOP_SIZE)
    
    Y_recarr=(np.abs(stft_recarr))**2
    
    start_time = time.time()
    m,n=Y_recarr.shape
    final_matrix = np.zeros((m,n))
    count=0

    for i in range(0,m-99):
        for j in range(0,n-99):
            mat = Y_recarr[i:i+100,j:j+100]
            r,c, = np.where(mat == np.amax(mat))
            final_matrix[(r+i),(j+c)] = 1

    print(final_matrix)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    mycursor=mydb.cursor()
    message="SELECT song_id FROM ref where Song_name=%s"
    mycursor.execute(message,sub)
    songid=mycursor.fetchall()
    print((songid[0])[0])
    
    mycursor=mydb.cursor()
    hashes=list([])
    for i in range(10,m-10):
        for j in range(n-100):
            if final_matrix[i][j]==1:
                ll,ul=j+10,j+100
                lb,ub=i-10,i+10
                for y in range(lb,ub+1):
                    for x in range(ll,ul+1):
                        if final_matrix[y][x]==1:
                            hashvalue=hashlib.sha224(f"{str(i)}|{str(y)}|{str(x-j)}".encode('utf-8')).hexdigest()
                            value=[hashvalue,(songid[0])[0],i]
                            prompt="INSERT INTO songsource(hashval_,song_id,offset_) VALUES (%s, %s, %s)"
                            mycursor.execute(prompt,value)
                            mydb.commit()
    print("--- %s seconds ---" % (time.time() - start_time))                 

In [4]:
if choice==2:
    filename = input('Enter the name of the recording')+'.wav'
    chunk = 1024
    FORMAT = pyaudio.paInt16
    channels = 1
    sample_rate = 44100
    record_seconds = 5
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    output=True,
                    frames_per_buffer=chunk)
    frames = []
    print("Recording...")
    for i in range(int(44100 / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)
    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b"".join(frames))
    wf.close()
    
    start_time = time.time()
    
    recarr,sr=librosa.load(filename)
    def ftransform(signal,title,sr,f_ratio):
        ft=np.fft.fft(signal)
        mag_spec=np.abs(ft)
    
    ftransform(recarr,filename,sr,0.5)
    
    FRAME_SIZE=2048
    HOP_SIZE=512
    
    stft_recarr=librosa.stft(recarr,n_fft=FRAME_SIZE,hop_length=HOP_SIZE)
    
    Y_recarr=(np.abs(stft_recarr))**2

    m,n=Y_recarr.shape
    final_matrix = np.zeros((m,n))

    for i in range(0,m-99):
        for j in range(0,n-99):
            mat = Y_recarr[i:i+100,j:j+100]
            r,c, = np.where(mat == np.amax(mat))
            final_matrix[(r+i),(j+c)] = 1
    
    mydb = mysql.connector.connect(
      host="localhost",
      user="<Enter database here>",
      passwd="<Enter password here>",
      database="songsdb"
    )

    mycursor=mydb.cursor()
    hashes=list([])
    for i in range(10,m-10):
        for j in range(n-100):
            if final_matrix[i][j]==1:
                ll,ul=j+10,j+100
                lb,ub=i-10,i+10
                for y in range(lb,ub+1):
                    for x in range(ll,ul+1):
                        if final_matrix[y][x]==1:
                            hashvalue=hashlib.sha224(f"{str(i)}|{str(y)}|{str(x-j)}".encode('utf-8')).hexdigest()
                            value=[hashvalue,i]
                            prompt="INSERT INTO rechashes (hashes, recoffset) VALUES (%s, %s)"
                            mycursor.execute(prompt,value)
                            mydb.commit()         
    
    mycursor=mydb.cursor()
    message="SELECT song_id,hashval_,offset_ FROM songsource"
    mycursor.execute(message)
    lamb=mycursor.fetchall()

    mycursor=mydb.cursor()
    message="SELECT hashes,recoffset FROM rechashes"
    mycursor.execute(message)
    chi=mycursor.fetchall()
    
    lo=[{},{},{},{},{},{},{},{},{}]
    for mem,n in chi:
        emp=list([])
        for songid,ele,dboff in lamb:
            if mem==ele:
                emp.append((songid,dboff))
        for fri in emp:
            di=lo[fri[0]-1]
            ite=0
            for word in di:
                if (fri[1]-n) == word:
                    di[word]+=1
                    ite+=1
            if ite==0:
                di[(fri[1]-n)]=1
    max=0
    reqnum=0
    for terms in lo:
        reqnum+=1
        final_di=terms
        for k in final_di:
            if final_di[k]>max:
                max=final_di[k]
                snum=reqnum
    pass_str="SELECT Song_name FROM ref WHERE song_id= %s "
    pass_val=[snum]
    mycursor.execute(pass_str,pass_val)
    finalres=(mycursor.fetchall())
    print("The recording is from the song",(finalres[0])[0])
    
    mycursor.execute("TRUNCATE TABLE rechashes")
    os.remove(filename)
    
    print("--- %s seconds ---" % (time.time() - start_time))

In [5]:
quit()